# ORGANISATION DES DONNEES

In [1]:
from utils.io import update_dict, gen_dated_id, dataframe_viewer, gen_geodf_geom, data_merger, data_validation, \
data_slicer, replicate_values, collect_measure, collect_time_data, gen_id_from_ech, na_col_drop, na_line_drop, col_ren, \
dble_col_drop, dict_viewer

from utils.config import DEFAULT_POL_LEXICON, POL_NAMES_MODEL 
from difflib import get_close_matches

import re, os
import numpy as np
import geopandas as gpd
import pandas as pd
import datetime as dtm
import matplotlib.pyplot as plt
from definitions import ROOT_DIR

In [2]:
def compute_BH_length(df, id_col='ID', length_col_name='Long_for', top_col='Intv_top', base_col='Intv_base', verbose=False):
    
    if length_col_name in df.columns:
        raise(NameError(f'{length_col_name} is already in columns. Give another name'))
    
    for i in df.index:
        try:
            float(df.loc[i, top_col])
        except ValueError:
            df.loc[i, top_col] = np.nan

        try:
            float(df.loc[i, base_col])
        except ValueError:
            df.loc[i, base_col] = np.nan

    df[top_col] = df[top_col].astype('float64')
    df[base_col] = df[base_col].astype('float64')

    # compute length based on litho_top and litho_base
    id_list = []

    for i in df.index:
        id_ = df.loc[i,id_col]
        
        if verbose : print(i, id_, df.loc[i, top_col], df.loc[i, base_col])
        if id_ not in id_list:
            id_list.append(id_)
            if isinstance(id_, str):
                sql_id = f"{id_}"
            elif isinstance(id_, float) or isinstance(id_, int):
                sql_id = id_
                
            tmp = df[df[id_col] == sql_id]
            
            if verbose : print(len(tmp))
            #if len(tmp) > 0:
            df.loc[tmp.index, length_col_name] = float(max(tmp[base_col])) - float(min(tmp[top_col]))
    
    df.drop(index=df.query(f'{base_col}.isnull() and {top_col}.isnull()').index, inplace=True)
    df.insert(df.columns.to_list().index(id_col)+1, length_col_name, df.pop(length_col_name))
    #df.reset_index(drop=True, inplace=True)
    

### Creation du répertoire de sauvegarde

In [3]:
save_dir = ROOT_DIR + '/CF_data/Result_traitem/organisation/'

In [4]:
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

### Definition de variables usuelles

In [5]:
MEAS_NAMES_MODEL = {'Fraction   2000 µm':'Fract_2000µ', 'Fraction   63 µm':'Fract_63µ', 'Fraction   45 µm':'Fract_45µ', 'Fraction   16 µm':'Fract_16µ', 
                    'Fraction   2 µm':'Fract_2µ', 'Fraction 2 mm':'Fract_2', 'Fraction +2 mm':'Fract_2+', 'Fract_2':'Fract_2', 'Fract_2+':'Fract_2+', 
                    'Mat. organique':'MO', 'Mat. sèche':'MS', 'Argile':'Fract_arg', 'Fraction argileuse':'Fract_arg'}

In [6]:
POL_NAMES_MODEL = {**POL_NAMES_MODEL, **MEAS_NAMES_MODEL}

In [7]:
params_kw = ['O_diss','Niv_eau', 'temp', '^T$', '^CE$', 'pH$', 'ORP']
meas_kw_col = ['O_diss','pH','CE','ORP','Niv_eau_pz','Niv_eau_sol','Temp']
sufx = ['sup', 'prof', 'inf', '/\dM(\*)?']
prefx = ['eau forage ']
id_reg = '\s*(?P<id>(?:^canne |Piezair )*\w*\d+\w*)\s*'
pollutants_names = list(set(list(DEFAULT_POL_LEXICON.abbreviations.keys()) + list(POL_NAMES_MODEL.values())))

In [8]:
bh_cols = list(set(['Date_for','ID','X','Y','Z','Zsol','Type','Long_for','Long_pz','Sect_crep','Long_pz_sol','Ht_pz_sol',
           'Diam_for','Diam_int_pz','Diam_ext_pz','Ht_chbre','Refus','Societe','Zone','Sous_zone','Etude','Method','Resp_chantier',
           'Emplacement','ID_date','Rmq']))

mes_cols = list(set(['Date_mes','ID','ID_ech','X','Y','Z','Zsol','pH_H2O', 'Temp_pH_H2O', 'Temp_pH_CaCl2','pH_CaCl2','Temp_pH_KCl',
            'pH_KCl','Residu_perte_feu','Fract_arg','Fract_min_2µ','Fract_min_50µ','Fract_min_2','Temp_pH_mes',
            'pH_H20', 'Fract_min_2µ', 'Fract_min_50µ', 'Fract_min_2', 'pH_KCl', 'Temp_pH_mes', 'pH_H20', 'sulfures_tot''N_Kjdl','Temp_CE','Temp_pH','Nappe','Rmq','Fract_2000µ','Fract_63µ','Fract_45µ','Fract_16µ',
            'Fract_2µ','Temp_ech', 'Periode'] + meas_kw_col + list(MEAS_NAMES_MODEL.values())))

eqp_cols = list(set(list(set(['Date_for','ID','X','Y','Z','Zsol','Type_equip','Equip_base','Equip_top','Rmq']))))

litho_cols = list(set(['Date_for','ID','ID_ech','X','Y','Z','Zsol','Long_for','Litho_top','Litho_base','Intv_top','Intv_base',
              'Description','Rmq']))

an_cols = list(set(['ID','X','Y','Z','Zsol','Date_ech','ID_ech','Type_ech','Ech_top','Ech_base','Intv_top','Intv_base',
           'Description','Nappe','Organo','Intensite', 'Min_organo', 'Max_organo', 'Polluant',
           'Surnageant','Sousnageant','Caractere','Opacite','Rmq'] + pollutants_names))

ukw_cols = list(set(['Date_for','ID','X','Y','Z','Zsol','Type','Long_for','Method','Societe','Rmq']))

cols_dict = {'borehole': bh_cols, 'measure': mes_cols, 'lithology': litho_cols, 'analysis': an_cols, 
 'equipement': eqp_cols, 'unknown': ukw_cols}

In [9]:
bh_crit = ['ID','X','Y','Z','Zsol','Type','Long_for','Long_pz','Diam_for','Diam_int_pz','Diam_ext_pz']

mes_crit = ['ID','ID_ech','Date_mes'] + meas_kw_col

eqp_crit = ['Type_equip','Equip_base','Equip_top']

litho_crit = ['Litho_top','Litho_base','Intv_top','Intv_base','Description']

an_crit = ['ID_ech','Type_ech','Organo','Surnageant','Sousnageant'] + list(DEFAULT_POL_LEXICON.abbreviations.keys()) 

ukw_crit = ['ID','X','Y','Z','Zsol','Long_for','Type']

crit_dict = {'borehole': bh_crit, 'measure': mes_crit, 'lithology': litho_crit, 'analysis': an_crit, 
 'equipement': eqp_crit, 'unknown': ukw_crit}

variables utilisées par jeu de données
================================
- bh 	: 	forages (simple ou piezo)
- equip	:	equipements d'un forage (outils, méthodes utilisés, ...)
- ukw	:	objets physiques indéterminés
- litho :	descriptions lithologiques
- an 	: 	analyses de contaminants sur des échantillons (sol, eau)
- mes	:	mesures de propriétés sur des échantillons (sol, eau), de paramètres hydrochimiques, ...


# ---------------------------------------------------------

In [10]:
# initialisation des variables
_df = pd.DataFrame()
source_bh, source_eqp, source_ukw = _df, _df, _df
source_litho, source_an, source_mes = _df, _df, _df

print(f'source_bh: {len(source_bh)} ; source_eqp: {len(source_eqp)} ; source_uknw: {len(source_ukw)} ; '
      f'source_litho: {len(source_litho)} ; source_an: {len(source_an)} ; source_mes: {len(source_mes)}')

source_bh: 0 ; source_eqp: 0 ; source_uknw: 0 ; source_litho: 0 ; source_an: 0 ; source_mes: 0


## 4-profondeur de contact campagne de forages octobre 2019.xlsx

* **Sheet : 'Feuil1'**

In [11]:
tmp_dir= save_dir + 'Prof_contact_sol_forage/'
sheet='Feuil1'

In [12]:
df = pd.read_excel(ROOT_DIR + '/CF_data/Data_UMONS/ouvrages/profondeur de contact campagne de forages octobre 2019.xlsx', 
                   sheet_name='Feuil1', skiprows=2)
df=na_line_drop(df,0)
df=na_col_drop(df,1)
df.replace(r'<|>','', inplace=True, regex=True)
df.replace(r'-$',np.nan, inplace=True, regex=True)

dataframe_viewer(df)

Rows : 8, columns : 5


interactive(children=(IntSlider(value=8, description='rows', max=8, min=8, readout=False), IntSlider(value=5, …

In [13]:
df.rename(columns={'n°forage ':'ID','profondeur(m)':'Long_for','x':'X', 'y':'Y', 'z':'Z'}, inplace=True)
df['Type']='Forage' # type is not defined clearly in data
df['ID']=df['ID'].apply(lambda x: 'F'+str(x).replace('.0',''))

bh=df

In [14]:
source_bh=bh

In [15]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir), os.makedirs(tmp_dir+'source_merge/')
    
#eqp.to_csv(tmp_dir+sheet+'_Equipments.csv', index=False)
#mes.to_csv(tmp_dir+sheet+'_Measures.csv', index=False)
bh.to_csv(tmp_dir+sheet+'_Boreholes.csv', index=False)
#an.to_csv(tmp_dir+sheet+'_Samples.csv', index=False)
#ukw.to_csv(tmp_dir+sheet+'_Unknown.csv', index=False)
#litho.to_csv(tmp_dir+sheet+'_Lithologies.csv', index=False)

source_bh.to_csv(tmp_dir+'source_merge/source_Boreholes.csv', index=False) #all Boreholes data in the source
#source_mes.to_csv(tmp_dir+'source_merge/source_Measures.csv', index=False) #all Measures data in the source
#source_eqp.to_csv(tmp_dir+'source_merge/source_Equipments.csv', index=False)
#source_an.to_csv(tmp_dir+'source_merge/source_Samples.csv', index=False)
#source_ukw.to_csv(tmp_dir+'source_merge/source_Unknown.csv', index=False)
#source_litho.to_csv(tmp_dir+'source_merge/source_Lithologies.csv', index=False)

print(f'source_bh: {len(source_bh)} ; source_eqp: {len(source_eqp)} ; source_uknw: {len(source_ukw)} ; '
      f'source_litho: {len(source_litho)} ; source_an: {len(source_an)} ; source_mes: {len(source_mes)}')

source_bh: 8 ; source_eqp: 0 ; source_uknw: 0 ; source_litho: 0 ; source_an: 0 ; source_mes: 0


#### ======================================================================================

In [16]:
# initialisation des variables
_df = pd.DataFrame()
source_bh, source_eqp, source_ukw = _df, _df, _df
source_litho, source_an, source_mes = _df, _df, _df

print(f'source_bh: {len(source_bh)} ; source_eqp: {len(source_eqp)} ; source_uknw: {len(source_ukw)} ; '
      f'source_litho: {len(source_litho)} ; source_an: {len(source_an)} ; source_mes: {len(source_mes)}')

source_bh: 0 ; source_eqp: 0 ; source_uknw: 0 ; source_litho: 0 ; source_an: 0 ; source_mes: 0


## 5-Forages_Pilote_Decoupe.xlsx

* **Sheet : 'leve'**

In [17]:
tmp_dir= save_dir + 'Forage_Pilote/'
sheet='leve_Z_elect_pos'

In [18]:
df = pd.read_excel(ROOT_DIR + '/CF_data/Data_UMONS/geometrie_electrodes_et_sondes/Forages_Pilote_Decoupe.xlsx', 
                   sheet_name='leve')#, skiprows=0)
df=na_line_drop(df,0)
df=na_col_drop(df,1)
df.replace(r'<|>','', inplace=True, regex=True)
df.replace(r'-$',np.nan, inplace=True, regex=True)

dataframe_viewer(df, rows=5)

Columns dropped :['Unnamed: 10']

Rows : 72, columns : 11


interactive(children=(IntSlider(value=5, description='rows', max=72, min=5, readout=False), IntSlider(value=11…

In [19]:
df.rename(columns={'Ref_puits':'ID','Niveau mesuré':'Z_mes', 'Niveau corrigé':'Z','Z_diff [m] repere_local':'Diff_Z_local',
                   'long_fin [m]':'Long_for','Pos_Inox_#1 [m]':'Pos_Inox_#1', 'Unnamed: 11':'Rmq',
                   'Pos_Inox_#6 [m]':'Pos_Inox_#6', 'Pos_Impol_#3 [m]':'Pos_Impol_#3'}, inplace=True)

In [20]:
df['Type']='Forage' # type is not defined clearly in data
df['ID']=df['ID'].apply(lambda x: 'F'+str(x).replace('.0',''))

elc = df[['ID','Pos_Inox_#6', 'Pos_Impol_#3']] # 'ID' is for boreholes
bh = df[['ID','Z','Diff_Z_local','Long_for', 'Type']]# Z_local origin = 145.5 [m]

In [21]:
dataframe_viewer(df, rows=3, un_val=['ID','ID_ech'])

Rows : 72, columns : 12, Unique values on cols: {'ID': 72, 'ID_ech': 'NA'}


interactive(children=(IntSlider(value=3, description='rows', max=72, min=3, readout=False), IntSlider(value=12…

In [22]:
source_bh = bh
source_elc = elc

In [23]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir), os.makedirs(tmp_dir+'source_merge/') 

bh.to_csv(tmp_dir+sheet+'_Boreholes.csv', index=False)
elc.to_csv(tmp_dir+sheet+'_Electrodes.csv', index=False)

source_bh.to_csv(tmp_dir+'source_merge/source_Boreholes.csv', index=False)
source_elc.to_csv(tmp_dir+'source_merge/source_Electrodes.csv', index=False)

print(f'source_bh:{len(source_bh)} ; source_elect:{len(source_elc)} ;')

source_bh:72 ; source_elect:72 ;


#### ======================================================================================

In [24]:
# initialisation des variables
_df = pd.DataFrame()
source_bh, source_eqp, source_ukw = _df, _df, _df
source_litho, source_an, source_mes = _df, _df, _df

print(f'source_bh: {len(source_bh)} ; source_eqp: {len(source_eqp)} ; source_uknw: {len(source_ukw)} ; '
      f'source_litho: {len(source_litho)} ; source_an: {len(source_an)} ; source_mes: {len(source_mes)}')

source_bh: 0 ; source_eqp: 0 ; source_uknw: 0 ; source_litho: 0 ; source_an: 0 ; source_mes: 0


## 6-Liste XY investigations.xlsx
* **Sheet : 'SOL_EAU'**

In [25]:
tmp_dir= save_dir + 'Liste_XY/'
sheet='Sol_Eau'

In [26]:
df = pd.read_excel(ROOT_DIR + '/CF_data/Data_UMONS/Contamination/Doc_SITEREM/Etude de caracterisation. SITEREM - 2011/Documents '
                   'supplémentaires/Liste XY investigations.xlsx', sheet_name='SOL')#, skiprows=4)
df['Type_ech']='Sol'
df.rename(columns={'N°':'ID_ech'}, inplace=True)

df1 = pd.read_excel(ROOT_DIR + '/CF_data/Data_UMONS/Contamination/Doc_SITEREM/Etude de caracterisation. SITEREM - 2011/Documents '
                   'supplémentaires/Liste XY investigations.xlsx', sheet_name='EAU PR')#, skiprows=4)
df1['Type_ech']='Eau'
df1['Nappe']='Socle'
df1.rename(columns={'N°':'ID_ech'}, inplace=True)

df2 = pd.read_excel(ROOT_DIR + '/CF_data/Data_UMONS/Contamination/Doc_SITEREM/Etude de caracterisation. SITEREM - 2011/Documents '
                   'supplémentaires/Liste XY investigations.xlsx', sheet_name='EAU RB')#, skiprows=4)
df2['Type_ech']='Eau'
df2['Nappe']='remblais'
df2.rename(columns={'N°':'ID_ech'}, inplace=True)

df3 = pd.read_excel(ROOT_DIR + '/CF_data/Data_UMONS/Contamination/Doc_SITEREM/Etude de caracterisation. SITEREM - 2011/Documents '
                   'supplémentaires/Liste XY investigations.xlsx', sheet_name='EAU ALL')#, skiprows=4)
df3['Type_ech']='Eau'
df3['Nappe']='Alluvions'
df3.rename(columns={'N°':'ID_ech'}, inplace=True)

In [27]:
dataframe_viewer(df, rows=3, un_val=['ID','ID_ech'])

Rows : 134, columns : 4, Unique values on cols: {'ID': 'NA', 'ID_ech': 134}


interactive(children=(IntSlider(value=3, description='rows', max=134, min=3, readout=False), IntSlider(value=4…

In [28]:
mdf, conflict_df=data_merger(df1, df, 'outer', 'ID_ech')

In [29]:
mdf, conflict_df=data_merger(mdf, df2, 'outer', 'ID_ech')

In [30]:
mdf, conflict_df=data_merger(mdf, df3, 'outer', 'ID_ech')

Conflict values present. Please resolve this manually !


In [31]:
dataset = mdf
data_validation(overall_data=dataset, conflict_data=conflict_df, index_col='index', pass_col='ID', 
                valid_dict={'Nappe_x':list(conflict_df.index)})

if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)
mdf = dataset.copy()

all conflicts have been fixed!


In [32]:
an = mdf
source_an = an
#source_an.insert(0,'ID', source_an.pop('ID_ech'))

In [33]:
source_an = gen_id_from_ech(source_an, suffixes=sufx, prefixes=prefx, capture_regex=id_reg)

In [34]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir), os.makedirs(tmp_dir+'source_merge/')
    
#eqp.to_csv(tmp_dir+sheet+'_Equipments.csv', index=False)
#mes.to_csv(tmp_dir+sheet+'_Measures.csv', index=False)
#bh.to_csv(tmp_dir+sheet+'_Boreholes.csv', index=False)
an.to_csv(tmp_dir+sheet+'_Samples.csv', index=False)
#ukw.to_csv(tmp_dir+sheet+'_Unknown.csv', index=False)
#litho.to_csv(tmp_dir+sheet+'_Lithologies.csv', index=False)

#source_bh.to_csv(tmp_dir+'source_merge/source_Boreholes.csv', index=False) #all Boreholes data in the source
#source_mes.to_csv(tmp_dir+'source_merge/source_Measures.csv', index=False) #all Measures data in the source
#source_eqp.to_csv(tmp_dir+'source_merge/source_Equipments.csv', index=False)
source_an.to_csv(tmp_dir+'source_merge/source_Samples.csv', index=False)
#source_ukw.to_csv(tmp_dir+'source_merge/source_Unknown.csv', index=False)
#source_litho.to_csv(tmp_dir+'source_merge/source_Lithologies.csv', index=False)

print(f'source_bh: {len(source_bh)} ; source_eqp: {len(source_eqp)} ; source_uknw: {len(source_ukw)} ; '
      f'source_litho: {len(source_litho)} ; source_an: {len(source_an)} ; source_mes: {len(source_mes)}')

source_bh: 0 ; source_eqp: 0 ; source_uknw: 0 ; source_litho: 0 ; source_an: 254 ; source_mes: 0


#### ======================================================================================

In [35]:
# initialisation des variables
_df = pd.DataFrame()
source_bh, source_eqp, source_ukw = _df, _df, _df
source_litho, source_an, source_mes = _df, _df, _df

print(f'source_bh: {len(source_bh)} ; source_eqp: {len(source_eqp)} ; source_uknw: {len(source_ukw)} ; '
      f'source_litho: {len(source_litho)} ; source_an: {len(source_an)} ; source_mes: {len(source_mes)}')

source_bh: 0 ; source_eqp: 0 ; source_uknw: 0 ; source_litho: 0 ; source_an: 0 ; source_mes: 0


## 7-Résultats phase 1_MEMORIS.xls
* **Sheet : 'Résult SOL'**

In [36]:
tmp_dir= save_dir + 'Phase_1_Memoris/'
sheet='Result_sol'

In [37]:
df = pd.read_excel(ROOT_DIR + '/CF_data/Data_UMONS/Contamination/Résultats et mesures Siterem/résultats phase 1/'
                   'Résultats phase 1_MEMORIS.xls', sheet_name='Résult SOL', skiprows=4)
df=na_line_drop(df,0)
df=na_col_drop(df,1)
df.replace(r'<|>','', inplace=True, regex=True)
df.replace(r'-$',np.nan, inplace=True, regex=True)

dataframe_viewer(df, rows=5)

1 NaN lines dropped
Columns dropped :['Unnamed: 0', 'Unnamed: 2']

Rows : 135, columns : 35


/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


interactive(children=(IntSlider(value=5, description='rows', max=135, min=5, readout=False), IntSlider(value=1…

In [38]:
ech_df=df.loc[:35]
an=df.loc[36:]

In [39]:
an.loc[0.5] = df.loc[0] # put data on first line
an = an.sort_index().reset_index(drop=True)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [40]:
ech_df=ech_df.transpose()
ech_df.reset_index(drop=True, inplace=True)
ech_df=col_ren(ech_df, 1)

In [41]:
ech_df.drop(list(range(5)), axis=0, inplace=True)
ech_df.reset_index(drop=True, inplace=True)
ech_df=na_col_drop(ech_df,1)
ech_df=na_line_drop(ech_df,3)
ech_df.reset_index(drop=True, inplace=True)

Columns dropped :['col_1', "Nom / description d'échantillon", 'Date de prélèvement', "Nature de l'étude (*)", 'Terrain', 'Epaisseur de remblais', 'Epaisseur alluvions', "Nature de l'observation organoleptique", 'LOCALISATION/AFFECTATION(S) - USAGE(S)', 'zone', 'parcelle (selon dénom.)', "Type d'usage de fait actuel", "Type d'usage de fait futur", "Type d'usage utilisé pour comparer les résultats", 'Lieux de prélèvement', 'X Lambert', 'Y Lambert', 'DESCRIPTION SOMMAIRE', 'Type de recouvrement de surface (***)', 'Sol en place (S : souillé, NS : non souillé)', 'Remblais de terre (S : souillé, NS : non souillé)', 'Remblais technique (S : souillé, NS : non souillé)', 'Déchet', 'Matières organiques', 'GRANULOMETRIE', 'Fraction argileuse']



In [42]:
name=['ID_ech', 'Description','Organo', 'Long_for', 'Refus','Ech_top', 'Ech_base', 'MS','Fract_2','Fract_2+']
ech_df=col_ren(ech_df, name=name, mode=1)

In [43]:
for i in range(len(ech_df['Description'])):
    x = ech_df.loc[i,'Description']
    if x=='R': ech_df.loc[i,'Description']='Remblais'
    elif x=='L': ech_df.loc[i,'Description']='Limons'
    elif x=='A': ech_df.loc[i,'Description']='Argiles'
    elif x=='S': ech_df.loc[i,'Description']='Sables'

ech_df['Refus']=ech_df['Refus'].apply(lambda x: 'x' if not re.search('x|X', str(x)) else '')
ech_df.insert(1,'Type_ech','Sol')

In [44]:
an=an.transpose()
an.reset_index(drop=True, inplace=True)

In [45]:
an=col_ren(an, 1)

In [46]:
an.replace(r'<|>','', inplace=True, regex=True)
an.replace(r'-',np.nan, inplace=True, regex=True)
an.rename(columns={'col_0':'ID_ech', 'col_34':'phénanthrène', 'col_63':'EOX'}, inplace=True)

In [47]:
an.drop(list(range(5)), axis=0, inplace=True)
an.reset_index(drop=True, inplace=True)
an=na_col_drop(an,1)
an.insert(1,'Type_ech','Sol')

Columns dropped :['METAUX LOURDS', 'CYANURES', 'COMPOSES AROMATIQUES VOLATILS', 'BTEX totaux', 'PHENOLS', 'Indice phénol', 'HYDROCARBURES AROMATIQUES POLYCYCLIQUES', 'COMPOSES ORGANOHALOGENES VOLATILS', 'EOX', 'EOX (****)', 'HYDROCARBURES TOTAUX', 'fraction aromat. C6-C7', 'fraction aromat. C7-C8', 'fraction aromat. C8-C10', 'fraction aliphat. C5-C6', 'fraction aliphat. C6-C8', 'fraction aliphat. C8-C10', 'Fraction C35 - C40', 'Hydrocarbures totaux C10-C40', 'METHYL-TERT-BUTYL-ETHER', 'POLYCHLOROBIPHENYL (PCB)', 'PCB 28', 'PCB 52', 'PCB 101', 'PCB 118', 'PCB 138', 'PCB 153', 'PCB 180', 'PCB totaux (7)', 'F4/2M*', 'Teneur mesurée', 'Teneur mesurée (souligne/gras)', 'Teneur mesurée (gras/grisé)', "(*) CP = Campagne de prélèvements; Ex = Expertise; ES = Etude de sol; EO = Etude d'orientation; EC = Etude de caractérisation; SA = Suivi d'assainissement", '(**) L: limon, A: Argile, S: Sable, R: Remblai', '(***) ib : imperméable (béton) ; ih : imperméable hydrocarboné ; p : perméable (gravier

In [48]:
an = col_ren(an, name=POL_NAMES_MODEL, mode=1)#, verbose=True)


Possible new pollutants names:
['ID_ech', 'Type_ech', 'Cyanure (libre)', 'Cyanure (totaux)', 'cyanure (APE)', 'Fraction C5 - C8', 'Fraction C8 - C10', 'Fraction C16 - C21']


In [49]:
mdf, conflict_df = data_merger(ech_df, an, how='outer', on=['ID_ech'], dist_max=1., drop_skip_col=['index'])

In [50]:
an = gen_id_from_ech(mdf, suffixes=sufx, prefixes=prefx, capture_regex=id_reg)
an['ID'] = an['ID'].apply(lambda x: x+'M')

In [51]:
df_dict = data_slicer(an, cols_dict, crit_dict)

borehole: 29 ; measure: 29 ; lithology: 0 ; analysis: 29 ; equipement: 0 ; unknown: 0 ; 


In [52]:
dataframe_viewer(an, rows=10, un_val=['ID','ID_ech'])

Rows : 29, columns : 74, Unique values on cols: {'ID': 15, 'ID_ech': 29}


interactive(children=(IntSlider(value=10, description='rows', max=29, min=10, readout=False), IntSlider(value=…

In [53]:
source_an=an

In [54]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir), os.makedirs(tmp_dir+'source_merge/')
    
#eqp.to_csv(tmp_dir+sheet+'_Equipments.csv', index=False)
#mes.to_csv(tmp_dir+sheet+'_Measures.csv', index=False)
#bh.to_csv(tmp_dir+sheet+'_Boreholes.csv', index=False)
an.to_csv(tmp_dir+sheet+'_Samples.csv', index=False)
#ukw.to_csv(tmp_dir+sheet+'_Unknown.csv', index=False)
#litho.to_csv(tmp_dir+sheet+'_Lithologies.csv', index=False)

#source_bh.to_csv(tmp_dir+'source_merge/source_Boreholes.csv', index=False) #all Boreholes data in the source
#source_mes.to_csv(tmp_dir+'source_merge/source_Measures.csv', index=False) #all Measures data in the source
#source_eqp.to_csv(tmp_dir+'source_merge/source_Equipments.csv', index=False)
source_an.to_csv(tmp_dir+'source_merge/source_Samples.csv', index=False)
#source_ukw.to_csv(tmp_dir+'source_merge/source_Unknown.csv', index=False)
#source_litho.to_csv(tmp_dir+'source_merge/source_Lithologies.csv', index=False)

print(f'source_bh: {len(source_bh)} ; source_eqp: {len(source_eqp)} ; source_uknw: {len(source_ukw)} ; '
      f'source_litho: {len(source_litho)} ; source_an: {len(source_an)} ; source_mes: {len(source_mes)}')

source_bh: 0 ; source_eqp: 0 ; source_uknw: 0 ; source_litho: 0 ; source_an: 29 ; source_mes: 0


* **Sheet : 'Résult EAU'**

In [55]:
tmp_dir= save_dir + 'Phase_1_Memoris/'
sheet='Result_eau'

In [56]:
df = pd.read_excel(ROOT_DIR + '/CF_data/Data_UMONS/Contamination/Résultats et mesures Siterem/résultats phase 1/'
                   'Résultats phase 1_MEMORIS.xls', sheet_name='Résult EAU', skiprows=4)
df=na_line_drop(df,0)
df=na_col_drop(df,1)
df.replace(r'<|>','', inplace=True, regex=True)
df.replace(r'-$',np.nan, inplace=True, regex=True)
dataframe_viewer(df, rows=5)

1 NaN lines dropped
Columns dropped :['Unnamed: 0', 'Unnamed: 2']

Rows : 136, columns : 23


/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


interactive(children=(IntSlider(value=5, description='rows', max=136, min=5, readout=False), IntSlider(value=1…

In [57]:
ech_df=df.loc[:32]
an=df.loc[33:]

In [58]:
an.loc[0.5] = df.loc[0] # put data on first line
an = an.sort_index().reset_index(drop=True)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [59]:
ech_df=ech_df.transpose()
ech_df.reset_index(drop=True, inplace=True)
ech_df=col_ren(ech_df, 1)

Double columns' name found :
[('DESCRIPTION SOMMAIRE', 2), ('Prof. arrêt du forage', 2)]


In [60]:
ech_df['CE']=ech_df['CE'].apply(lambda x: pd.to_numeric(x)/1000 
                                  if re.search('^\d+', str(x)) and not pd.isnull(x) else np.nan)

In [61]:
ech_df.drop(list(range(5)), axis=0, inplace=True)
ech_df.reset_index(drop=True, inplace=True)

In [62]:
ech_df=dble_col_drop(ech_df)

column(s) dropped: ['21:DESCRIPTION SOMMAIRE', '27:Prof. arrêt du forage']


In [63]:
ech_df=na_col_drop(ech_df,1)
ech_df=na_line_drop(ech_df,3)
ech_df.reset_index(drop=True, inplace=True)

Columns dropped :['col_1', "Nature de l'étude (*)", 'Observations organoleptiques', 'LOCALISATION/AFFECTATION(S) - USAGE(S)', 'parcelle (selon dénom.)', "Type d'usage de fait actuel", "Type d'usage de fait futur", "Type d'usage utilisé pour comparer les résultats", 'Lieux de prélèvement', 'DESCRIPTION SOMMAIRE', 'Type de recouvrement de surface (**)', 'Sol en place (S : souillé, NS : non souillé)', 'Remblais de terre (S : souillé, NS : non souillé)', 'Remblais technique (S : souillé, NS : non souillé)', 'Déchet', 'PARAMETRES PHYSICO-CHIMIQUES ', 'ORP']



In [64]:
name=['ID_ech','Date_ech','Num_maille','Affectation','X','Y','Zsol','Long_for','Prof_crep','Long_pz',
      'Niv_eau_sol','pH','CE','Temp']
ech_df=col_ren(ech_df, name=name, mode=1)
ech_df.insert(1,'Type_ech','Eau')

In [65]:
ech_df['Prof_crep'].replace('\[|\]','', regex=True, inplace=True)
for i in range(len(ech_df)):
    c=ech_df.loc[i,'Prof_crep']
    ech_df.loc[i,'Equip_top']=c.split('-')[0]
    ech_df.loc[i,'Equip_base']=c.split('-')[1]

ech_df['Type_equip'] = 'Crepine'
ech_df.drop(columns=['Prof_crep'], inplace=True)

In [66]:
#ech_df['ID_ech'].replace('Canne ', 'Can', inplace=True, regex=True)
ech_df['ID_ech'].replace('\n', ' ', inplace=True, regex=True)

In [67]:
an=an.transpose()
an.reset_index(drop=True, inplace=True)

In [68]:
an=col_ren(an, 1)

Double columns' name found :
[('Teneur mesurée', 2)]


In [69]:
an.replace(r'<|>','', inplace=True, regex=True)
an.replace(r'-',np.nan, inplace=True, regex=True)
an.rename(columns={an.columns[0]:'ID_ech', 'col_43':'phénanthrène'}, inplace=True)

In [70]:
an.drop(list(range(5)), axis=0, inplace=True)
an.reset_index(drop=True, inplace=True)

In [71]:
an.drop(columns=an.columns[[-6,-5]], axis=1, inplace=True)
an=na_col_drop(an,1)
an.insert(1,'Type_ech','Eau')

Columns dropped :['METAUX LOURDS', 'CYANURES', 'COMPOSES AROMATIQUES VOLATILS', 'PHENOLS', 'crésols (total)', 'CHLOROPHENOLS', '2-chlorophénol', 'monochlorophénol total', 'dichlorophénol total', '2,4,5-trichlorophénol', '2,4,6-trichlorophénol', 'trichlorophénol total', '2,3,4,6- tétrachlorophénol', 'tétrachlorophénol total', 'pentachlorophénol', 'HYDROCARBURES AROMATIQUES POLYCYCLIQUES', 'COMPOSES ORGANOCHLORES VOLATILS', 'CHLOROBENZENES', 'monochlorobenzène', '1,3-dichlorobenzène', '1,2-dichlorobenzène', '1,4-Dichlorobenzène', '1,2,4,5- et 1,2,3,5-tétrachlorobenzènes', 'pentachlorobenzène', 'hexachlorobenzène', 'HYDROCARBURES TOTAUX', 'METHYL-TERT-BUTYL-ETHER', 'POLYCHLOROBIPHENYLS (PCB)', 'PCB totaux (7)', 'Teneur mesurée (souligne/gras)', 'Teneur mesurée (gras/grisé)', "(*) CP = Campagne de prélèvements; Ex = Expertise; ES = Etude de sol; EO = Etude d'orientation; EC = Etude de caractérisation; SA = Suivi d'assainissement", '(**) ib : imperméable (béton) ; ih : imperméable hydrocarb

In [72]:
an=col_ren(an, name=POL_NAMES_MODEL, mode=1)


Possible new pollutants names:
['ID_ech', 'Type_ech', 'cyanure (APE)', 'Tétrachloroéthylène ', 'fraction aromat. C6-C7', 'fraction aromat. C7-C8', 'fraction aromat. C8-C10', 'fraction aliphat. C6-C8', 'fraction aliphat. C8-C10', 'Fraction C5 - C8', 'Fraction C8 - C10', 'Fraction C16 - C21']


In [73]:
mdf, conflict_df = data_merger(ech_df, an, how='outer', on=['ID_ech'], dist_max=1., drop_skip_col=['index'])

In [74]:
df = gen_id_from_ech(mdf, suffixes=sufx, prefixes=prefx, capture_regex=id_reg)

In [75]:
df['Type'] = 'Piezo'
df['Date_mes'] = df['Date_ech']

In [76]:
df_dict = data_slicer(df, cols_dict, crit_dict)

borehole: 17 ; measure: 17 ; lithology: 0 ; analysis: 17 ; equipement: 17 ; unknown: 0 ; 

Not used columns:
 ['Num_maille', 'Affectation']


In [77]:
ukw = df_dict['unknown']
bh = df_dict['borehole']

bh = bh.drop(index=ukw.index)
ukw.drop_duplicates(['ID'], inplace=True)
ukw.reset_index(drop=True, inplace=True)
bh.drop_duplicates(['ID'], inplace=True)
if 'X' in bh.columns: bh = bh.query('ID==ID and X==X')
bh.reset_index(drop=True, inplace=True)

mes = df_dict['measure']
an = df_dict['analysis']
litho = df_dict['lithology']
eqp = df_dict['equipement']

print(f'borehole: {len(bh)} ; measure: {len(mes)} ; lithology: {len(litho)} ; analysis: {len(an)} ; ' 
      f'equipement: {len(eqp)} ; unknown: {len(ukw)}')

borehole: 14 ; measure: 17 ; lithology: 0 ; analysis: 17 ; equipement: 17 ; unknown: 0


In [78]:
source_an, conflict_df = data_merger(source_an, an, how='outer', on=['ID_ech', 'Type_ech'], dist_max=1., drop_skip_col=['index'])

In [79]:
source_bh = bh
source_mes = mes
source_eqp = eqp

In [80]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir), os.makedirs(tmp_dir+'source_merge/')
    
eqp.to_csv(tmp_dir+sheet+'_Equipments.csv', index=False)
mes.to_csv(tmp_dir+sheet+'_Measures.csv', index=False)
bh.to_csv(tmp_dir+sheet+'_Boreholes.csv', index=False)
an.to_csv(tmp_dir+sheet+'_Samples.csv', index=False)
#ukw.to_csv(tmp_dir+sheet+'_Unknown.csv', index=False)
#litho.to_csv(tmp_dir+sheet+'_Lithologies.csv', index=False)

source_bh.to_csv(tmp_dir+'source_merge/source_Boreholes.csv', index=False) #all Boreholes data in the source
source_mes.to_csv(tmp_dir+'source_merge/source_Measures.csv', index=False) #all Measures data in the source
source_eqp.to_csv(tmp_dir+'source_merge/source_Equipments.csv', index=False)
source_an.to_csv(tmp_dir+'source_merge/source_Samples.csv', index=False)
#source_ukw.to_csv(tmp_dir+'source_merge/source_Unknown.csv', index=False)
#source_litho.to_csv(tmp_dir+'source_merge/source_Lithologies.csv', index=False)

print(f'source_bh: {len(source_bh)} ; source_eqp: {len(source_eqp)} ; source_uknw: {len(source_ukw)} ; '
      f'source_litho: {len(source_litho)} ; source_an: {len(source_an)} ; source_mes: {len(source_mes)}')

source_bh: 14 ; source_eqp: 17 ; source_uknw: 0 ; source_litho: 0 ; source_an: 46 ; source_mes: 17


#### ======================================================================================

In [81]:
# initialisation des variables
_df = pd.DataFrame()
source_bh, source_eqp, source_ukw = _df, _df, _df
source_litho, source_an, source_mes = _df, _df, _df

print(f'source_bh: {len(source_bh)} ; source_eqp: {len(source_eqp)} ; source_uknw: {len(source_ukw)} ; '
      f'source_litho: {len(source_litho)} ; source_an: {len(source_an)} ; source_mes: {len(source_mes)}')

source_bh: 0 ; source_eqp: 0 ; source_uknw: 0 ; source_litho: 0 ; source_an: 0 ; source_mes: 0


## 8-Résultats phase 2_MEMORIS.xls
* **Sheet : 'Résult SOL'**

In [82]:
tmp_dir= save_dir + 'Phase_2_Memoris/'
sheet='Result_SOL'

In [83]:
df = pd.read_excel(ROOT_DIR + '/CF_data/Data_UMONS/Contamination/Résultats et mesures Siterem/résultats phase 2/'
                   'Résultats phase 2_MEMORIS.xls', sheet_name='Résult SOL', skiprows=4)
df=na_line_drop(df,0)
df=na_col_drop(df,1)
df.replace(r'<|>','', inplace=True, regex=True)
df.replace(r'-$',np.nan, inplace=True, regex=True)
dataframe_viewer(df, rows=5)

1 NaN lines dropped
Columns dropped :['Unnamed: 0', 'Unnamed: 2']

Rows : 135, columns : 31


/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


interactive(children=(IntSlider(value=5, description='rows', max=135, min=5, readout=False), IntSlider(value=1…

In [84]:
ech_df=df.loc[:35]
an=df.loc[36:]

In [85]:
an.loc[0.5] = df.loc[1] # put data on first line
an = an.sort_index().reset_index(drop=True)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [86]:
ech_df=ech_df.transpose()
ech_df.reset_index(drop=True, inplace=True)
ech_df=col_ren(ech_df, 1)

In [87]:
ech_df.drop(list(range(5)), axis=0, inplace=True)
ech_df.reset_index(drop=True, inplace=True)
ech_df=na_col_drop(ech_df,1)
ech_df=na_line_drop(ech_df,3)
ech_df.reset_index(drop=True, inplace=True)

Columns dropped :['col_0', "Nom / description d'échantillon", "Nature de l'étude (*)", 'Terrain', 'Epaisseur de remblais', 'Epaisseur alluvions', "Nature de l'observation organoleptique", 'LOCALISATION/AFFECTATION(S) - USAGE(S)', 'zone', 'parcelle (selon dénom.)', "Type d'usage de fait actuel", "Type d'usage de fait futur", "Type d'usage utilisé pour comparer les résultats", 'Lieux de prélèvement', 'X Lambert', 'Y Lambert', 'DESCRIPTION SOMMAIRE', 'Type de recouvrement de surface (***)', 'Sol en place (S : souillé, NS : non souillé)', 'Remblais de terre (S : souillé, NS : non souillé)', 'Remblais technique (S : souillé, NS : non souillé)', 'Déchet', 'Matières organiques', 'GRANULOMETRIE', 'Fraction argileuse']



In [88]:
name=['ID_ech', 'Date_ech', 'Description','Organo', 'Long_for', 'Refus','Ech_top', 'Ech_base', 'MS','Fract_2','Fract_2+']
ech_df=col_ren(ech_df, name=name, mode=1)

In [89]:
set(ech_df['Description'])

{'L', 'LA', 'LS', 'R'}

In [90]:
for i in range(len(ech_df['Description'])):
    x = ech_df.loc[i,'Description']
    if x=='R': ech_df.loc[i,'Description']='Remblais'
    elif x=='L': ech_df.loc[i,'Description']='Limons'
    elif x=='LA': ech_df.loc[i,'Description']='Limons et argiles'
    elif x=='LS': ech_df.loc[i,'Description']='Limons et sables'

ech_df['Refus']=ech_df['Refus'].apply(lambda x: 'x' if not re.search('#', str(x)) else '')
ech_df.insert(1,'Type_ech','Sol')

In [91]:
an=an.transpose()
an.reset_index(drop=True, inplace=True)

In [92]:
an=col_ren(an, 1)

In [93]:
an.replace(r'<|>','', inplace=True, regex=True)
an.replace(r'-',np.nan, inplace=True, regex=True)
an.rename(columns={'col_0':'ID_ech', 'col_34':'phénanthrène'}, inplace=True)

In [94]:
an.drop(list(range(5)), axis=0, inplace=True)
an=na_col_drop(an,1)
an.insert(1,'Type_ech','Sol')

Columns dropped :['METAUX LOURDS', 'CYANURES', 'COMPOSES AROMATIQUES VOLATILS', 'BTEX totaux', 'PHENOLS', 'Phénol', 'Indice phénol', 'HYDROCARBURES AROMATIQUES POLYCYCLIQUES', 'COMPOSES ORGANOHALOGENES VOLATILS', '1,1-Dichloroéthane', '1,2-Dichloroéthane', '1,1-dichloroéthène', 'Cis-1,2-dichloroéthène', 'Trans 1,2-dichloroéthylène', 'Dichlorométhane', 'Totaux (cis,trans) 1,2-dichloroéthènes', '1,2-dichloropropane', 'Tétrachloroéthylène', 'Tétrachlorométhane', '1,1,1-Trichloroéthane', '1,1,2-Trichloroéthane', 'Trichloroéthylène', 'Chloroforme', 'Chlorure de vinyle', 'col_63', 'EOX (****)', 'HYDROCARBURES TOTAUX', 'Fraction C35 - C40', 'Hydrocarbures totaux C10-C40', 'METHYL-TERT-BUTYL-ETHER', 'MTBE', 'POLYCHLOROBIPHENYL (PCB)', 'PCB 28', 'PCB 52', 'PCB 101', 'PCB 118', 'PCB 138', 'PCB 153', 'PCB 180', 'PCB totaux (7)', 'F4/2M*', 'Teneur mesurée', 'Teneur mesurée (souligne/gras)', 'Teneur mesurée (gras/grisé)', "(*) CP = Campagne de prélèvements; Ex = Expertise; ES = Etude de sol; EO = E

In [95]:
an=col_ren(an, name=POL_NAMES_MODEL, mode=1)


Possible new pollutants names:
['ID_ech', 'Type_ech', 'Cyanure (libre)', 'Cyanure (totaux)', 'cyanure (APE)', 'fraction aromat. C6-C7', 'fraction aromat. C7-C8', 'fraction aromat. C8-C10', 'fraction aliphat. C6-C8', 'fraction aliphat. C8-C10', 'Fraction C5 - C8', 'Fraction C8 - C10', 'Fraction C16 - C21']


In [96]:
mdf, conflict_df = data_merger(ech_df, an, how='outer', on=['ID_ech'], dist_max=1., drop_skip_col=['index'])

In [97]:
df = gen_id_from_ech(mdf, suffixes=sufx, prefixes=prefx, capture_regex=id_reg)

In [98]:
df['Type'] = 'Forage'

In [99]:
df_dict = data_slicer(df, cols_dict, crit_dict)

borehole: 25 ; measure: 25 ; lithology: 0 ; analysis: 25 ; equipement: 0 ; unknown: 0 ; 


In [100]:
ukw = df_dict['unknown']
bh = df_dict['borehole']

bh = bh.drop(index=ukw.index)
ukw.drop_duplicates(['ID'], inplace=True)
ukw.reset_index(drop=True, inplace=True)
bh.drop_duplicates(['ID'], inplace=True)
if 'X' in bh.columns: bh = bh.query('ID==ID and X==X')
bh.reset_index(drop=True, inplace=True)

mes = df_dict['measure']
an = df_dict['analysis']
litho = df_dict['lithology']
eqp = df_dict['equipement']

print(f'borehole: {len(bh)} ; measure: {len(mes)} ; lithology: {len(litho)} ; analysis: {len(an)} ; ' 
      f'equipement: {len(eqp)} ; unknown: {len(ukw)}')

borehole: 12 ; measure: 25 ; lithology: 0 ; analysis: 25 ; equipement: 0 ; unknown: 0


In [101]:
source_bh = bh
source_an = an

In [102]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir), os.makedirs(tmp_dir+'source_merge/')
    
#eqp.to_csv(tmp_dir+sheet+'_Equipments.csv', index=False)
#mes.to_csv(tmp_dir+sheet+'_Measures.csv', index=False)
bh.to_csv(tmp_dir+sheet+'_Boreholes.csv', index=False)
an.to_csv(tmp_dir+sheet+'_Samples.csv', index=False)
#ukw.to_csv(tmp_dir+sheet+'_Unknown.csv', index=False)
#litho.to_csv(tmp_dir+sheet+'_Lithologies.csv', index=False)

source_bh.to_csv(tmp_dir+'source_merge/source_Boreholes.csv', index=False) #all Boreholes data in the source
#source_mes.to_csv(tmp_dir+'source_merge/source_Measures.csv', index=False) #all Measures data in the source
#source_eqp.to_csv(tmp_dir+'source_merge/source_Equipments.csv', index=False)
source_an.to_csv(tmp_dir+'source_merge/source_Samples.csv', index=False)
#source_ukw.to_csv(tmp_dir+'source_merge/source_Unknown.csv', index=False)
#source_litho.to_csv(tmp_dir+'source_merge/source_Lithologies.csv', index=False)

print(f'source_bh: {len(source_bh)} ; source_eqp: {len(source_eqp)} ; source_uknw: {len(source_ukw)} ; '
      f'source_litho: {len(source_litho)} ; source_an: {len(source_an)} ; source_mes: {len(source_mes)}')

source_bh: 12 ; source_eqp: 0 ; source_uknw: 0 ; source_litho: 0 ; source_an: 25 ; source_mes: 0


* **Sheet : 'Résult EAU'**

In [103]:
tmp_dir= save_dir + 'Phase_2_Memoris/'
sheet='Result_eau'

In [104]:
df = pd.read_excel(ROOT_DIR + '/CF_data/Data_UMONS/Contamination/Résultats et mesures Siterem/résultats phase 2/'
                   'Résultats phase 2_MEMORIS.xls', sheet_name='Résult EAU', skiprows=4)
df=na_line_drop(df,0)
df=na_col_drop(df,1)
df.replace(r'<|>','', inplace=True, regex=True)
df.replace(r'-$',np.nan, inplace=True, regex=True)

dataframe_viewer(df, rows=5)

Columns dropped :['Unnamed: 0', 'Unnamed: 2']

Rows : 138, columns : 17


interactive(children=(IntSlider(value=5, description='rows', max=138, min=5, readout=False), IntSlider(value=1…

In [105]:
ech_df=df.loc[:32]
an=df.loc[33:]

In [106]:
an.loc[0.5] = df.loc[1] # put data on first line
an = an.sort_index().reset_index(drop=True)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [107]:
ech_df=ech_df.transpose()
ech_df.reset_index(drop=True, inplace=True)
ech_df=col_ren(ech_df, 1)

Double columns' name found :
[('DESCRIPTION SOMMAIRE', 2), ('Prof. arrêt du forage', 2)]


In [108]:
ech_df['CE']=ech_df['CE'].apply(lambda x: pd.to_numeric(x)/1000 
                                  if re.search('^\d+', str(x)) and not pd.isnull(x) else np.nan)

In [109]:
ech_df.drop(list(range(5)), axis=0, inplace=True)
ech_df.reset_index(drop=True, inplace=True)

In [110]:
ech_df=dble_col_drop(ech_df)

column(s) dropped: ['21:DESCRIPTION SOMMAIRE', '27:Prof. arrêt du forage']


In [111]:
ech_df=na_col_drop(ech_df,1)
ech_df=na_line_drop(ech_df,3)
ech_df.reset_index(drop=True, inplace=True)

Columns dropped :['Nom du piézomètre', "Nature de l'étude (*)", 'Observations organoleptiques', 'LOCALISATION/AFFECTATION(S) - USAGE(S)', 'Numéro de maille', 'parcelle (selon dénom.)', "Type d'usage de fait actuel", "Type d'usage de fait futur", "Type d'usage utilisé pour comparer les résultats", 'Lieux de prélèvement', 'DESCRIPTION SOMMAIRE', 'Type de recouvrement de surface (**)', 'Sol en place (S : souillé, NS : non souillé)', 'Remblais de terre (S : souillé, NS : non souillé)', 'Remblais technique (S : souillé, NS : non souillé)', 'Déchet', 'PARAMETRES PHYSICO-CHIMIQUES ', 'ORP']



In [112]:
name=['ID_ech', 'Date_ech','Affectation','X', 'Y','Zsol', 'Long_for','Prof_crep','Long_pz_sol',
      'Niv_eau_sol','pH', 'CE', 'Temp']
ech_df=col_ren(ech_df, name=name, mode=1)
ech_df.insert(1,'Type_ech','Eau')

In [113]:
ech_df['ID_ech'].replace('\n', ' ', inplace=True, regex=True)

In [114]:
ech_df['Prof_crep'].replace('\[|\]','', regex=True, inplace=True)
for i in range(len(ech_df)):
    c=ech_df.loc[i,'Prof_crep']
    ech_df.loc[i,'Equip_top']=c.split('-')[0]
    ech_df.loc[i,'Equip_base']=c.split('-')[1]
    
ech_df.drop(columns=['Prof_crep'], inplace=True)
ech_df['Type_equip'] = 'Crepine'
ech_df['Type']='Piezo'

In [115]:
an=an.transpose()
an.reset_index(drop=True, inplace=True)

In [116]:
an=col_ren(an, 1)

Double columns' name found :
[('Teneur mesurée', 2)]


In [117]:
an.replace(r'<|>','', inplace=True, regex=True)
an.replace(r'-',np.nan, inplace=True, regex=True)
an.rename(columns={'col_0':'ID_ech', 'col_43':'phénanthrène'}, inplace=True)

In [118]:
an.drop(list(range(5)), axis=0, inplace=True)
an.reset_index(drop=True, inplace=True)

In [119]:
an.drop(columns=an.columns[[-6,-5]], axis=1, inplace=True)
an=na_col_drop(an,1)
an.insert(1,'Type_ech','Eau')

Columns dropped :['METAUX LOURDS', 'Chrome VI', 'CYANURES', 'COMPOSES AROMATIQUES VOLATILS', 'PHENOLS', 'crésols (total)', 'CHLOROPHENOLS', '2-chlorophénol', 'monochlorophénol total', 'dichlorophénol total', '2,4,5-trichlorophénol', '2,4,6-trichlorophénol', 'trichlorophénol total', '2,3,4,6- tétrachlorophénol', 'tétrachlorophénol total', 'pentachlorophénol', 'HYDROCARBURES AROMATIQUES POLYCYCLIQUES', 'COMPOSES ORGANOCHLORES VOLATILS', 'CHLOROBENZENES', 'monochlorobenzène', '1,3-dichlorobenzène', '1,2-dichlorobenzène', '1,4-Dichlorobenzène', '1,2,4,5- et 1,2,3,5-tétrachlorobenzènes', 'pentachlorobenzène', 'hexachlorobenzène', 'HYDROCARBURES TOTAUX', 'METHYL-TERT-BUTYL-ETHER', 'POLYCHLOROBIPHENYLS (PCB)', 'PCB totaux (7)', 'AUTRES ANALYSES ', 'Teneur mesurée (souligne/gras)', 'Teneur mesurée (gras/grisé)', "(*) CP = Campagne de prélèvements; Ex = Expertise; ES = Etude de sol; EO = Etude d'orientation; EC = Etude de caractérisation; SA = Suivi d'assainissement", '(**) ib : imperméable (bé

In [120]:
an=col_ren(an, name=POL_NAMES_MODEL, mode=1)


Possible new pollutants names:
['ID_ech', 'Type_ech', 'cyanure (APE)', 'Tétrachloroéthylène ', 'fraction aromat. C6-C7', 'fraction aromat. C7-C8', 'fraction aromat. C8-C10', 'fraction aliphat. C6-C8', 'fraction aliphat. C8-C10', 'Fraction C5 - C8', 'Fraction C8 - C10', 'Fraction C16 - C21']


In [121]:
mdf, conflict_df = data_merger(ech_df, an, how='outer', on=['ID_ech'], dist_max=1., drop_skip_col=['index'])

In [122]:
mdf['Type'] = 'Piezo'
mdf['Date_mes'] = mdf['Date_ech']

In [123]:
df = gen_id_from_ech(mdf, suffixes=sufx, prefixes=prefx)

In [124]:
df_dict = data_slicer(df, cols_dict, crit_dict)

borehole: 11 ; measure: 11 ; lithology: 0 ; analysis: 11 ; equipement: 11 ; unknown: 0 ; 

Not used columns:
 ['Affectation']


In [125]:
ukw = df_dict['unknown']
bh = df_dict['borehole']

bh = bh.drop(index=ukw.index)
ukw.drop_duplicates(['ID'], inplace=True)
ukw.reset_index(drop=True, inplace=True)
bh.drop_duplicates(['ID'], inplace=True)
if 'X' in bh.columns: bh = bh.query('ID==ID and X==X')
bh.reset_index(drop=True, inplace=True)

mes = df_dict['measure']
an = df_dict['analysis']
litho = df_dict['lithology']
eqp = df_dict['equipement']

print(f'borehole: {len(bh)} ; measure: {len(mes)} ; lithology: {len(litho)} ; analysis: {len(an)} ; ' 
      f'equipement: {len(eqp)} ; unknown: {len(ukw)}')

borehole: 10 ; measure: 11 ; lithology: 0 ; analysis: 11 ; equipement: 11 ; unknown: 0


In [126]:
source_bh, conflict_df = data_merger(source_bh, bh, how='outer', on=['ID'], dist_max=1., drop_skip_col=['index'])

In [127]:
source_an, conflict_df = data_merger(source_an, an, how='outer', on=['ID_ech', 'Type_ech'], dist_max=1., drop_skip_col=['index'])

In [128]:
source_mes = mes
source_eqp = eqp

In [129]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir), os.makedirs(tmp_dir+'source_merge/')
    
eqp.to_csv(tmp_dir+sheet+'_Equipments.csv', index=False)
mes.to_csv(tmp_dir+sheet+'_Measures.csv', index=False)
bh.to_csv(tmp_dir+sheet+'_Boreholes.csv', index=False)
an.to_csv(tmp_dir+sheet+'_Samples.csv', index=False)
#ukw.to_csv(tmp_dir+sheet+'_Unknown.csv', index=False)
#litho.to_csv(tmp_dir+sheet+'_Lithologies.csv', index=False)

source_bh.to_csv(tmp_dir+'source_merge/source_Boreholes.csv', index=False) #all Boreholes data in the source
source_mes.to_csv(tmp_dir+'source_merge/source_Measures.csv', index=False) #all Measures data in the source
source_eqp.to_csv(tmp_dir+'source_merge/source_Equipments.csv', index=False)
source_an.to_csv(tmp_dir+'source_merge/source_Samples.csv', index=False)
#source_ukw.to_csv(tmp_dir+'source_merge/source_Unknown.csv', index=False)
#source_litho.to_csv(tmp_dir+'source_merge/source_Lithologies.csv', index=False)

print(f'source_bh: {len(source_bh)} ; source_eqp: {len(source_eqp)} ; source_uknw: {len(source_ukw)} ; '
      f'source_litho: {len(source_litho)} ; source_an: {len(source_an)} ; source_mes: {len(source_mes)}')

source_bh: 22 ; source_eqp: 11 ; source_uknw: 0 ; source_litho: 0 ; source_an: 36 ; source_mes: 11


#### ======================================================================================

In [130]:
# initialisation des variables
_df = pd.DataFrame()
source_bh, source_eqp, source_ukw = _df, _df, _df
source_litho, source_an, source_mes = _df, _df, _df

print(f'source_bh: {len(source_bh)} ; source_eqp: {len(source_eqp)} ; source_uknw: {len(source_ukw)} ; '
      f'source_litho: {len(source_litho)} ; source_an: {len(source_an)} ; source_mes: {len(source_mes)}')

source_bh: 0 ; source_eqp: 0 ; source_uknw: 0 ; source_litho: 0 ; source_an: 0 ; source_mes: 0


## 9-Ensemble des résultats Memoris version Seafile.xls
* **Sheet : 'Résult SOL'**

In [131]:
tmp_dir= save_dir + 'Memoris_seafile/'
sheet='Result_SOL'

In [132]:
df = pd.read_excel(ROOT_DIR + '/CF_data/Data_UMONS/Contamination/Résultats et mesures Siterem/'
                   'Ensemble des résultats Memoris version Seafile.xls', sheet_name='Résult SOL', skiprows=4)
df=na_line_drop(df,0)
df=na_col_drop(df,1)
dataframe_viewer(df, rows=5)

2 NaN lines dropped
Columns dropped :['Unnamed: 0', 'Unnamed: 2']

Rows : 138, columns : 66


/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


interactive(children=(IntSlider(value=5, description='rows', max=138, min=5, readout=False), IntSlider(value=1…

In [133]:
ech_df=df.loc[:37]
an=df.loc[38:]

In [134]:
an.loc[0.5] = df.loc[0] # put data on first line
an = an.sort_index().reset_index(drop=True)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [135]:
ech_df=ech_df.transpose()
ech_df.reset_index(drop=True, inplace=True)
ech_df=col_ren(ech_df, 1)

In [136]:
ech_df.drop(list(range(5)), axis=0, inplace=True)
ech_df.reset_index(drop=True, inplace=True)
ech_df=na_col_drop(ech_df,1)
ech_df=na_line_drop(ech_df,3)
ech_df.reset_index(drop=True, inplace=True)

Columns dropped :['col_1', "Nom / description d'échantillon", "Nature de l'étude (*)", 'Terrain', 'Epaisseur de remblais', 'Epaisseur alluvions', "Nature de l'observation organoleptique", 'LOCALISATION/AFFECTATION(S) - USAGE(S)', 'zone', 'parcelle (selon dénom.)', "Type d'usage de fait actuel", "Type d'usage de fait futur", "Type d'usage utilisé pour comparer les résultats", 'Lieux de prélèvement', 'X Lambert', 'Y Lambert', 'DESCRIPTION SOMMAIRE', 'Type de recouvrement de surface (***)', 'Sol en place (S : souillé, NS : non souillé)', 'Remblais de terre (S : souillé, NS : non souillé)', 'Remblais technique (S : souillé, NS : non souillé)', 'Déchet', 'Matières organiques', 'GRANULOMETRIE', 'Fraction argileuse']



In [137]:
ech_df.drop(columns=ech_df.columns[[-3,-4]], axis=1, inplace=True)

In [138]:
name=['ID_ech', 'Date_ech', 'Description','Organo', 'Long_for', 'Refus','Ech_top', 'Ech_base', 'MS','Fract_2','Fract_2+']
ech_df=col_ren(ech_df, name=name, mode=1)

In [139]:
set(ech_df['Description'])

{'L', 'LA', 'LS', 'R', 'R '}

In [140]:
for i in ech_df.index:
    x = ech_df.loc[i,'Description']
    if x=='R' or x=='R ': ech_df.loc[i,'Description']='Remblais'
    elif x=='L': ech_df.loc[i,'Description']='Limons'
    elif x=='LA': ech_df.loc[i,'Description']='Limons et argiles'
    elif x=='LS': ech_df.loc[i,'Description']='Limons et sables'

ech_df['Refus']=ech_df['Refus'].apply(lambda x: 'x' if not re.search('#', str(x)) else '')
ech_df.insert(1,'Type_ech','Sol')

In [141]:
an=an.transpose()
an.reset_index(drop=True, inplace=True)

In [142]:
an=col_ren(an, 1)

In [143]:
an.replace(r'<|>','', inplace=True, regex=True)
an.replace(r'-',np.nan, inplace=True, regex=True)

In [144]:
an=dble_col_drop(an)

column(s) dropped: []


In [145]:
an.drop(list(range(5)), axis=0, inplace=True)
an.reset_index(drop=True, inplace=True)
an=na_col_drop(an,1)

Columns dropped :['METAUX LOURDS', 'CYANURES', 'COMPOSES AROMATIQUES VOLATILS', 'BTEX totaux', 'PHENOLS', 'Indice phénol', 'HYDROCARBURES AROMATIQUES POLYCYCLIQUES', 'COMPOSES ORGANOHALOGENES VOLATILS', 'col_63', 'EOX (****)', 'HYDROCARBURES TOTAUX', 'Hydrocarbures totaux C10-C40', 'METHYL-TERT-BUTYL-ETHER', 'POLYCHLOROBIPHENYL (PCB)', 'PCB 28', 'PCB 52', 'PCB 101', 'PCB 118', 'PCB 138', 'PCB 153', 'PCB 180', 'PCB totaux (7)', 'F4/2M*', 'Teneur mesurée', 'Teneur mesurée (souligne/gras)', 'Teneur mesurée (gras/grisé)', "(*) CP = Campagne de prélèvements; Ex = Expertise; ES = Etude de sol; EO = Etude d'orientation; EC = Etude de caractérisation; SA = Suivi d'assainissement", '(**) L: limon, A: Argile, S: Sable, R: Remblai', '(***) ib : imperméable (béton) ; ih : imperméable hydrocarboné ; p : perméable (gravier, fissuré,…) ; tvh : terres végétation haute ; tvb : terres végétation basse ', '(****) 3 mg/kg = Seuil limite défini dans le GREO ', "(1) l'échantillon n'a pas pu être extrait ni 

In [146]:
an.rename(columns={'col_0':'ID_ech', 'col_34':'phénanthrène'}, inplace=True)
an.insert(1,'Type_ech','Sol')

In [147]:
an=col_ren(an, name=POL_NAMES_MODEL, mode=1)


Possible new pollutants names:
['ID_ech', 'Type_ech', 'Cyanure (libre)', 'Cyanure (totaux)', 'cyanure (APE)', 'Fraction C5 - C8', 'Fraction C8 - C10', 'Fraction C16 - C21']


In [148]:
mdf, conflict_df = data_merger(ech_df, an, how='outer', on=['ID_ech'], dist_max=1., drop_skip_col=['index'])

In [149]:
mdf['Type'] = 'Forage'

In [150]:
df = gen_id_from_ech(mdf, suffixes=sufx, prefixes=prefx, capture_regex=id_reg)

In [151]:
df_dict = data_slicer(df, cols_dict, crit_dict)

borehole: 60 ; measure: 60 ; lithology: 0 ; analysis: 60 ; equipement: 0 ; unknown: 0 ; 


In [152]:
ukw = df_dict['unknown']
bh = df_dict['borehole']

bh = bh.drop(index=ukw.index)
ukw.drop_duplicates(['ID'], inplace=True)
ukw.reset_index(drop=True, inplace=True)
bh.drop_duplicates(['ID'], inplace=True)
if 'X' in bh.columns: bh = bh.query('ID==ID and X==X')
bh.reset_index(drop=True, inplace=True)

mes = df_dict['measure']
an = df_dict['analysis']
litho = df_dict['lithology']
eqp = df_dict['equipement']

print(f'borehole: {len(bh)} ; measure: {len(mes)} ; lithology: {len(litho)} ; analysis: {len(an)} ; ' 
      f'equipement: {len(eqp)} ; unknown: {len(ukw)}')

borehole: 33 ; measure: 60 ; lithology: 0 ; analysis: 60 ; equipement: 0 ; unknown: 0


In [153]:
source_bh = bh
source_an = an

In [154]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir), os.makedirs(tmp_dir+'source_merge/')
    
#eqp.to_csv(tmp_dir+sheet+'_Equipments.csv', index=False)
#mes.to_csv(tmp_dir+sheet+'_Measures.csv', index=False)
bh.to_csv(tmp_dir+sheet+'_Boreholes.csv', index=False)
an.to_csv(tmp_dir+sheet+'_Samples.csv', index=False)
#ukw.to_csv(tmp_dir+sheet+'_Unknown.csv', index=False)
#litho.to_csv(tmp_dir+sheet+'_Lithologies.csv', index=False)

source_bh.to_csv(tmp_dir+'source_merge/source_Boreholes.csv', index=False) #all Boreholes data in the source
#source_mes.to_csv(tmp_dir+'source_merge/source_Measures.csv', index=False) #all Measures data in the source
#source_eqp.to_csv(tmp_dir+'source_merge/source_Equipments.csv', index=False)
source_an.to_csv(tmp_dir+'source_merge/source_Samples.csv', index=False)
#source_ukw.to_csv(tmp_dir+'source_merge/source_Unknown.csv', index=False)
#source_litho.to_csv(tmp_dir+'source_merge/source_Lithologies.csv', index=False)

print(f'source_bh: {len(source_bh)} ; source_eqp: {len(source_eqp)} ; source_uknw: {len(source_ukw)} ; '
      f'source_litho: {len(source_litho)} ; source_an: {len(source_an)} ; source_mes: {len(source_mes)}')

source_bh: 33 ; source_eqp: 0 ; source_uknw: 0 ; source_litho: 0 ; source_an: 60 ; source_mes: 0


* **Sheet : 'Résult EAU'**

In [155]:
tmp_dir= save_dir + 'Memoris_seafile/'
sheet='Result_eau'

In [156]:
df = pd.read_excel(ROOT_DIR + '/CF_data/Data_UMONS/Contamination/Résultats et mesures Siterem/'
                   'Ensemble des résultats Memoris version Seafile.xls', sheet_name='Résult EAU', skiprows=4)
df=na_line_drop(df,0)
df=na_col_drop(df,1)
df.replace(r'<|>','', inplace=True, regex=True)
df.replace(r'-$',np.nan, inplace=True, regex=True)

dataframe_viewer(df, rows=5)

4 NaN lines dropped
Columns dropped :['Unnamed: 0', 'Unnamed: 2']

Rows : 154, columns : 51


/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


interactive(children=(IntSlider(value=5, description='rows', max=154, min=5, readout=False), IntSlider(value=1…

In [157]:
ech_df=df.loc[:32]
an=df.loc[33:]

In [158]:
an.loc[0.5] = df.loc[0] # put data on first line
an = an.sort_index().reset_index(drop=True)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [159]:
ech_df=ech_df.transpose()
ech_df.reset_index(drop=True, inplace=True)
ech_df=col_ren(ech_df, 1)

Double columns' name found :
[('DESCRIPTION SOMMAIRE', 2), ('Prof. arrêt du forage', 2)]


In [160]:
ech_df['CE']=ech_df['CE'].apply(lambda x: pd.to_numeric(x)/1000 
                                  if re.search('^\d+', str(x)) and not pd.isnull(x) else np.nan)

In [161]:
ech_df.drop(list(range(5)), axis=0, inplace=True)
ech_df.reset_index(drop=True, inplace=True)

In [162]:
ech_df=dble_col_drop(ech_df)

column(s) dropped: ['21:DESCRIPTION SOMMAIRE', '27:Prof. arrêt du forage']


In [163]:
ech_df=na_col_drop(ech_df,1)
ech_df=na_line_drop(ech_df,3)
ech_df.reset_index(drop=True, inplace=True)

Columns dropped :['col_1', "Nature de l'étude (*)", 'Observations organoleptiques', 'LOCALISATION/AFFECTATION(S) - USAGE(S)', 'parcelle (selon dénom.)', "Type d'usage de fait actuel", "Type d'usage de fait futur", "Type d'usage utilisé pour comparer les résultats", 'Lieux de prélèvement', 'DESCRIPTION SOMMAIRE', 'Type de recouvrement de surface (**)', 'Sol en place (S : souillé, NS : non souillé)', 'Remblais de terre (S : souillé, NS : non souillé)', 'Remblais technique (S : souillé, NS : non souillé)', 'Déchet', 'PARAMETRES PHYSICO-CHIMIQUES ', 'ORP']



In [164]:
ech_df.drop(columns=ech_df.columns[[2]], axis=2, inplace=True)

In [165]:
name=['ID_ech', 'Date_ech','Affectation','X', 'Y','Zsol', 'Long_for','Prof_crep','Long_pz_sol', 
      'Niv_eau_sol','pH', 'CE', 'Temp']
ech_df=col_ren(ech_df, name=name, mode=1)
ech_df.insert(1,'Type_ech','Eau')

In [166]:
ech_df['Prof_crep'].replace('\[|\]','', regex=True, inplace=True)
for i in range(len(ech_df)):
    c=ech_df.loc[i,'Prof_crep']
    ech_df.loc[i,'Equip_top']=c.split('-')[0]
    ech_df.loc[i,'Equip_base']=c.split('-')[1]
    
ech_df.drop(columns=['Prof_crep'], inplace=True)
ech_df['Type_equip'] = 'Crepine'

In [167]:
ech_df['ID_ech'].replace('\n', ' ', inplace=True, regex=True)

In [168]:
an=an.transpose()
an.reset_index(drop=True, inplace=True)

In [169]:
an=col_ren(an, 1)

Double columns' name found :
[('nitrite', 2), ('nitrate', 2), ('ammonium', 2), ('Teneur mesurée', 2)]


In [170]:
an.replace(r'<|>','', inplace=True, regex=True)
an.replace(r'-',np.nan, inplace=True, regex=True)
an.rename(columns={an.columns[0]:'ID_ech','Température pour mes. pH':'Temp_pH', 'col_43':'phénanthrène'}, inplace=True)

In [171]:
an.drop(list(range(5)), axis=0, inplace=True)
an.reset_index(drop=True, inplace=True)

In [172]:
an=dble_col_drop(an)

column(s) dropped: ['104:nitrite', '106:nitrate', '112:ammonium', '117:Teneur mesurée']


In [173]:
an.drop(columns=an.columns[[-6,-5]], axis=1, inplace=True)
an=na_col_drop(an,1)
an.insert(1,'Type_ech','Eau')

Columns dropped :['METAUX LOURDS', 'CYANURES', 'COMPOSES AROMATIQUES VOLATILS', 'PHENOLS', 'crésols (total)', 'CHLOROPHENOLS', '2-chlorophénol', 'monochlorophénol total', 'dichlorophénol total', '2,4,5-trichlorophénol', '2,4,6-trichlorophénol', 'trichlorophénol total', '2,3,4,6- tétrachlorophénol', 'tétrachlorophénol total', 'pentachlorophénol', 'HYDROCARBURES AROMATIQUES POLYCYCLIQUES', 'COMPOSES ORGANOCHLORES VOLATILS', 'CHLOROBENZENES', 'monochlorobenzène', '1,3-dichlorobenzène', '1,2-dichlorobenzène', '1,4-Dichlorobenzène', '1,2,4,5- et 1,2,3,5-tétrachlorobenzènes', 'pentachlorobenzène', 'hexachlorobenzène', 'HYDROCARBURES TOTAUX', 'METHYL-TERT-BUTYL-ETHER', 'POLYCHLOROBIPHENYLS (PCB)', 'PCB totaux (7)', 'AUTRES ANALYSES ', 'azote Kjeldahl', 'COMPOSES INORGANIQUES ', 'sulfures totaux', 'Teneur mesurée (souligne/gras)', 'Teneur mesurée (gras/grisé)', "(*) CP = Campagne de prélèvements; Ex = Expertise; ES = Etude de sol; EO = Etude d'orientation; EC = Etude de caractérisation; SA = S

In [174]:
an['ID_ech'].replace('\n', ' ', inplace=True, regex=True)

In [175]:
an=col_ren(an, name=POL_NAMES_MODEL, mode=1)


Possible new pollutants names:
['ID_ech', 'Type_ech', 'cyanure (APE)', 'Tétrachloroéthylène ', 'fraction aromat. C6-C7', 'fraction aromat. C7-C8', 'fraction aromat. C8-C10', 'fraction aliphat. C6-C8', 'fraction aliphat. C8-C10', 'Fraction C5 - C8', 'Fraction C8 - C10', 'Fraction C16 - C21', 'pH', 'Temp_pH', 'sulfites', 'sulfate', 'sulfures (libre)']


In [176]:
mdf, conflict_df = data_merger(ech_df, an, how='outer', on=['ID_ech'], dist_max=1., drop_skip_col=['index'])

In [177]:
mdf['Type'] = 'Piezo'
mdf['Date_mes'] = mdf['Date_ech']

In [178]:
df = gen_id_from_ech(mdf, suffixes=sufx, prefixes=prefx, capture_regex=id_reg)

In [179]:
dataframe_viewer(df, rows=3, un_val=['ID','ID_ech'])

Rows : 45, columns : 96, Unique values on cols: {'ID': 30, 'ID_ech': 45}


interactive(children=(IntSlider(value=3, description='rows', max=45, min=3, readout=False), IntSlider(value=12…

In [180]:
df_dict = data_slicer(df, cols_dict, crit_dict)

borehole: 45 ; measure: 45 ; lithology: 0 ; analysis: 45 ; equipement: 45 ; unknown: 0 ; 

Not used columns:
 ['Affectation']


In [181]:
ukw = df_dict['unknown']
bh = df_dict['borehole']

bh = bh.drop(index=ukw.index)
ukw.drop_duplicates(['ID'], inplace=True)
ukw.reset_index(drop=True, inplace=True)
bh.drop_duplicates(['ID'], inplace=True)
if 'X' in bh.columns: bh = bh.query('ID==ID and X==X')
bh.reset_index(drop=True, inplace=True)

mes = df_dict['measure']
an = df_dict['analysis']
litho = df_dict['lithology']
eqp = df_dict['equipement']

print(f'borehole: {len(bh)} ; measure: {len(mes)} ; lithology: {len(litho)} ; analysis: {len(an)} ; ' 
      f'equipement: {len(eqp)} ; unknown: {len(ukw)}')

borehole: 24 ; measure: 45 ; lithology: 0 ; analysis: 45 ; equipement: 45 ; unknown: 0


In [182]:
source_bh, conflict_df = data_merger(source_bh, bh, how='outer', on=['ID'], dist_max=1., drop_skip_col=['index'])

In [183]:
source_an, conflict_df = data_merger(source_an, an, how='outer', on=['ID_ech', 'Type_ech'], dist_max=1., drop_skip_col=['index'])

In [184]:
source_mes = mes
source_eqp = eqp

In [185]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir), os.makedirs(tmp_dir+'source_merge/')
    
eqp.to_csv(tmp_dir+sheet+'_Equipments.csv', index=False)
mes.to_csv(tmp_dir+sheet+'_Measures.csv', index=False)
bh.to_csv(tmp_dir+sheet+'_Boreholes.csv', index=False)
an.to_csv(tmp_dir+sheet+'_Samples.csv', index=False)
#ukw.to_csv(tmp_dir+sheet+'_Unknown.csv', index=False)
#litho.to_csv(tmp_dir+sheet+'_Lithologies.csv', index=False)

source_bh.to_csv(tmp_dir+'source_merge/source_Boreholes.csv', index=False) #all Boreholes data in the source
source_mes.to_csv(tmp_dir+'source_merge/source_Measures.csv', index=False) #all Measures data in the source
source_eqp.to_csv(tmp_dir+'source_merge/source_Equipments.csv', index=False)
source_an.to_csv(tmp_dir+'source_merge/source_Samples.csv', index=False)
#source_ukw.to_csv(tmp_dir+'source_merge/source_Unknown.csv', index=False)
#source_litho.to_csv(tmp_dir+'source_merge/source_Lithologies.csv', index=False)

print(f'source_bh: {len(source_bh)} ; source_eqp: {len(source_eqp)} ; source_uknw: {len(source_ukw)} ; '
      f'source_litho: {len(source_litho)} ; source_an: {len(source_an)} ; source_mes: {len(source_mes)}')

source_bh: 57 ; source_eqp: 45 ; source_uknw: 0 ; source_litho: 0 ; source_an: 105 ; source_mes: 45


#### ======================================================================================

In [186]:
# initialisation des variables
_df = pd.DataFrame()
source_bh, source_eqp, source_ukw = _df, _df, _df
source_litho, source_an, source_mes = _df, _df, _df

print(f'source_bh: {len(source_bh)} ; source_eqp: {len(source_eqp)} ; source_uknw: {len(source_ukw)} ; '
      f'source_litho: {len(source_litho)} ; source_an: {len(source_an)} ; source_mes: {len(source_mes)}')

source_bh: 0 ; source_eqp: 0 ; source_uknw: 0 ; source_litho: 0 ; source_an: 0 ; source_mes: 0


## 10-Résultats SOL container phyto t=0_décret sol.xls
* **Sheet : 'Résult SOL'**

In [187]:
tmp_dir= save_dir + 'Container_phyto/'
sheet='Result_SOL'

In [188]:
df = pd.read_excel(ROOT_DIR + '/CF_data/Data_UMONS/Contamination/Résultats et mesures Siterem/'
                   'Résultats SOL container phyto t=0_décret sol.xls', sheet_name='Résult SOL', skiprows=4)
df=na_line_drop(df,0)
df=na_col_drop(df,1)
df.replace(r'<|>','', inplace=True, regex=True)
df.replace(r'-$',np.nan, inplace=True, regex=True)

dataframe_viewer(df, rows=5)

2 NaN lines dropped
Columns dropped :['Unnamed: 0', 'Unnamed: 2', 'Unnamed: 16', 'Unnamed: 17']

Rows : 121, columns : 15


/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


interactive(children=(IntSlider(value=5, description='rows', max=121, min=5, readout=False), IntSlider(value=1…

In [189]:
ech_df=df.loc[:21]
an=df.loc[22:]

In [190]:
an.loc[0.5] = df.loc[0] # put data on first line
an = an.sort_index().reset_index(drop=True)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [191]:
ech_df=ech_df.transpose()
ech_df.reset_index(drop=True, inplace=True)
ech_df=col_ren(ech_df, 1)

Double columns' name found :
[('Autre zone suspecte investiguée', 2)]


In [192]:
ech_df=dble_col_drop(ech_df)

column(s) dropped: ['8:Autre zone suspecte investiguée']


In [193]:
ech_df.drop(list(range(5)), axis=0, inplace=True)
ech_df.reset_index(drop=True, inplace=True)
ech_df=na_col_drop(ech_df,2)
ech_df=na_line_drop(ech_df,3)
ech_df.reset_index(drop=True, inplace=True)

Columns dropped :['Matières organiques', 'SPP/zone suspecte invetiguée', 'Autre zone suspecte investiguée', 'Parcelle', 'X Lambert', 'Y Lambert', "Type d'étude (*)", 'Refus de forage', 'Terrain naturel/Remblai (**)', 'Organoleptique couleur suspecte', 'Organoleptique odeur intensité (***)', 'Organoleptique odeur type', 'GRANULOMETRIE']

4 NaN lines dropped


/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [194]:
ech_df.drop(columns=ech_df.columns[[-3]], axis=1, inplace=True)

In [195]:
name=['ID_ech', 'Ech_top', 'Ech_base','MS','Date_ech','Fract_2','Fract_2+']
ech_df=col_ren(ech_df, name=name, mode=1)
ech_df=ech_df.query('ID_ech==ID_ech')
ech_df.insert(1,'Type_ech','Sol')

In [196]:
an=an.transpose()
an.reset_index(drop=True, inplace=True)

In [197]:
an=col_ren(an, 1)

Double columns' name found :
[('Teneur mesurée', 4)]


In [198]:
an.replace(r'<|>','', inplace=True, regex=True)
an.replace(r'-',np.nan, inplace=True, regex=True)

In [199]:
an=dble_col_drop(an)

column(s) dropped: ['92:Teneur mesurée', '93:Teneur mesurée', '94:Teneur mesurée']


In [200]:
an.drop(list(range(5)), axis=0, inplace=True)
an.reset_index(drop=True, inplace=True)
an=na_col_drop(an,1)
an.rename(columns={an.columns[0]:'ID_ech',  'col_35':'phénanthrène'}, inplace=True)
an.insert(1,'Type_ech','Sol')

Columns dropped :['METAUX LOURDS', 'Cobalt', 'CYANURES', 'COMPOSES AROMATIQUES VOLATILS', 'BTEX totaux', 'PHENOLS', 'Indice phénol', 'HYDROCARBURES AROMATIQUES POLYCYCLIQUES', 'COMPOSES ORGANOHALOGENES VOLATILS', 'col_64', 'EOX (****)', 'HYDROCARBURES TOTAUX', 'METHYL-TERT-BUTYL-ETHER', 'POLYCHLOROBIPHENYL (PCB)', 'PCB 28', 'PCB 52', 'PCB 101', 'PCB 118', 'PCB 138', 'PCB 153', 'PCB 180', 'PCB totaux (7)', 'Teneur mesurée', "VR : Valeur de référence; VS : Valeur seuil; VI : Valeur d'intervention", "(*) RP : Rapport de prélèvements; ES : Etude de sol; EO : Etude d'orientation; EC : Etude de caractérisation; SA : Suivi d'assainissement; EF : Evaluation finale", '(**) TN : Terrain naturel; R : Remblai', "(***) - : Pas d'impression organoleptique; + : Impression organoleptique faible; ++ : Impression organoleptique forte", '(****) 3 mg/kg = Seuil limite défini dans le GREO ']



In [201]:
an=col_ren(an, name=POL_NAMES_MODEL, mode=1)


Possible new pollutants names:
['ID_ech', 'Type_ech', 'Cyanure (libre)', 'Cyanure (totaux)', 'cyanure (APE)', 'fraction aromat. C6-C7', 'fraction aromat. C7-C8', 'fraction aromat. C8-C10', 'fraction aliphat. C6-C8', 'fraction aliphat. C8-C10', 'Fraction C5 - C8', 'Fraction C8 - C10', 'Fraction C16 - C21']


In [202]:
mdf, conflict_df = data_merger(ech_df, an, how='outer', on=['ID_ech'], dist_max=1., drop_skip_col=['index'])

In [203]:
df_dict = data_slicer(mdf, cols_dict, crit_dict)

borehole: 0 ; measure: 0 ; lithology: 0 ; analysis: 6 ; equipement: 0 ; unknown: 0 ; 


In [204]:
ukw = df_dict['unknown']
bh = df_dict['borehole']

bh = bh.drop(index=ukw.index)
ukw.drop_duplicates(['ID'], inplace=True)
ukw.reset_index(drop=True, inplace=True)
bh.drop_duplicates(['ID'], inplace=True)
if 'X' in bh.columns: bh = bh.query('ID==ID and X==X')
bh.reset_index(drop=True, inplace=True)

mes = df_dict['measure']
an = df_dict['analysis']
litho = df_dict['lithology']
eqp = df_dict['equipement']

print(f'borehole: {len(bh)} ; measure: {len(mes)} ; lithology: {len(litho)} ; analysis: {len(an)} ; ' 
      f'equipement: {len(eqp)} ; unknown: {len(ukw)}')

borehole: 0 ; measure: 0 ; lithology: 0 ; analysis: 6 ; equipement: 0 ; unknown: 0


In [205]:
source_an = an

In [206]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir), os.makedirs(tmp_dir+'source_merge/')
    
#eqp.to_csv(tmp_dir+sheet+'_Equipments.csv', index=False)
#mes.to_csv(tmp_dir+sheet+'_Measures.csv', index=False)
#bh.to_csv(tmp_dir+sheet+'_Boreholes.csv', index=False)
an.to_csv(tmp_dir+sheet+'_Samples.csv', index=False)
#ukw.to_csv(tmp_dir+sheet+'_Unknown.csv', index=False)
#litho.to_csv(tmp_dir+sheet+'_Lithologies.csv', index=False)

#source_bh.to_csv(tmp_dir+'source_merge/source_Boreholes.csv', index=False) #all Boreholes data in the source
#source_mes.to_csv(tmp_dir+'source_merge/source_Measures.csv', index=False) #all Measures data in the source
#source_eqp.to_csv(tmp_dir+'source_merge/source_Equipments.csv', index=False)
source_an.to_csv(tmp_dir+'source_merge/source_Samples.csv', index=False)
#source_ukw.to_csv(tmp_dir+'source_merge/source_Unknown.csv', index=False)
#source_litho.to_csv(tmp_dir+'source_merge/source_Lithologies.csv', index=False)

print(f'source_bh: {len(source_bh)} ; source_eqp: {len(source_eqp)} ; source_uknw: {len(source_ukw)} ; '
      f'source_litho: {len(source_litho)} ; source_an: {len(source_an)} ; source_mes: {len(source_mes)}')

source_bh: 0 ; source_eqp: 0 ; source_uknw: 0 ; source_litho: 0 ; source_an: 6 ; source_mes: 0


* **Sheet : 'Paramètres agro.'**

In [207]:
tmp_dir= save_dir + 'Container_phyto/'
sheet='Param_agro'

In [208]:
df = pd.read_excel(ROOT_DIR + '/CF_data/Data_UMONS/Contamination/Résultats et mesures Siterem/'
                   'Résultats SOL container phyto t=0_décret sol.xls', sheet_name='Paramètres agro.', skiprows=4)
df=na_line_drop(df,0)
df=na_col_drop(df,1)
df.replace(r'<|>','', inplace=True, regex=True)
df.replace(r'-$',np.nan, inplace=True, regex=True)

dataframe_viewer(df, rows=5)

1 NaN lines dropped
Columns dropped :['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2']

Rows : 28, columns : 10


interactive(children=(IntSlider(value=5, description='rows', max=28, min=5, readout=False), IntSlider(value=10…

In [209]:
df=df.transpose()
df.reset_index(drop=True, inplace=True)
df=col_ren(df, 0)

Double columns' name found :
[('température pour mes. pH', 2)]


In [210]:
df.drop(list(range(1)), axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)

In [211]:
df=dble_col_drop(df)

column(s) dropped: ['22:température pour mes. pH']


In [212]:
df=na_col_drop(df,1)
df=na_line_drop(df,3)
df.reset_index(drop=True, inplace=True)

Columns dropped :['Matières organiques', 'GRANULOMETRIE', 'pH', 'Composés inorganiques ', 'Autres analyses chimiques ']

3 NaN lines dropped


/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [213]:
df.drop(columns=df.columns[[5,6]], axis=2, inplace=True)

In [214]:
name=['ID_ech','Ech_top','Ech_base','MS','Date_ech','MO','Residu_perte_feu','COT','Fract_arg','Fract_min_2µ', 
      'Fract_min_50µ', 'Fract_min_2', 'Fract_2', 'Fract_2+', 'pH_KCl','Temp_pH_mes', 'pH_H20', 'Sulfure_tot', 
      'Chlorure', 'N_Kjdl']
df=col_ren(df, name=name, mode=1)
df.insert(1,'Type_ech','Sol')

In [215]:
#mdf['Type'] = 'Forage'
df['Date_mes'] = df['Date_ech']

In [216]:
df_dict = data_slicer(df, cols_dict, crit_dict)

borehole: 0 ; measure: 5 ; lithology: 0 ; analysis: 5 ; equipement: 0 ; unknown: 0 ; 


In [217]:
ukw = df_dict['unknown']
bh = df_dict['borehole']

bh = bh.drop(index=ukw.index)
ukw.drop_duplicates(['ID'], inplace=True)
ukw.reset_index(drop=True, inplace=True)
bh.drop_duplicates(['ID'], inplace=True)
if 'X' in bh.columns: bh = bh.query('ID==ID and X==X')
bh.reset_index(drop=True, inplace=True)

mes = df_dict['measure']
an = df_dict['analysis']
litho = df_dict['lithology']
eqp = df_dict['equipement']

print(f'borehole: {len(bh)} ; measure: {len(mes)} ; lithology: {len(litho)} ; analysis: {len(an)} ; ' 
      f'equipement: {len(eqp)} ; unknown: {len(ukw)}')

borehole: 0 ; measure: 5 ; lithology: 0 ; analysis: 5 ; equipement: 0 ; unknown: 0


In [218]:
source_an, conflict_df = data_merger(source_an, an, how='outer', on=['ID_ech'], dist_max=1., drop_skip_col=['index'])

Conflict values present. Please resolve this manually !


In [219]:
dataset = source_an
data_validation(overall_data=dataset, conflict_data=conflict_df, index_col='index', pass_col='ID', 
                valid_dict={'Fract_2_x':list(conflict_df.index), 'Fract_2+_x':list(conflict_df.index),
                           'MS_x':list(conflict_df.index)})

if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)
source_an = dataset

all conflicts have been fixed!


In [220]:
source_mes = mes

In [221]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir), os.makedirs(tmp_dir+'source_merge/')
    
#eqp.to_csv(tmp_dir+sheet+'_Equipments.csv', index=False)
mes.to_csv(tmp_dir+sheet+'_Measures.csv', index=False)
#bh.to_csv(tmp_dir+sheet+'_Boreholes.csv', index=False)
an.to_csv(tmp_dir+sheet+'_Samples.csv', index=False)
#ukw.to_csv(tmp_dir+sheet+'_Unknown.csv', index=False)
#litho.to_csv(tmp_dir+sheet+'_Lithologies.csv', index=False)

#source_bh.to_csv(tmp_dir+'source_merge/source_Boreholes.csv', index=False) #all Boreholes data in the source
source_mes.to_csv(tmp_dir+'source_merge/source_Measures.csv', index=False) #all Measures data in the source
#source_eqp.to_csv(tmp_dir+'source_merge/source_Equipments.csv', index=False)
source_an.to_csv(tmp_dir+'source_merge/source_Samples.csv', index=False)
#source_ukw.to_csv(tmp_dir+'source_merge/source_Unknown.csv', index=False)
#source_litho.to_csv(tmp_dir+'source_merge/source_Lithologies.csv', index=False)

print(f'source_bh: {len(source_bh)} ; source_eqp: {len(source_eqp)} ; source_uknw: {len(source_ukw)} ; '
      f'source_litho: {len(source_litho)} ; source_an: {len(source_an)} ; source_mes: {len(source_mes)}')

source_bh: 0 ; source_eqp: 0 ; source_uknw: 0 ; source_litho: 0 ; source_an: 6 ; source_mes: 5
